In [ ]:
# Read in data
master_df = pd.read_csv(
    "data/synthetic/standard/clean_synthetic_standard_survey.standard_school_aggregate_responses.0000000010000.csv",
    encoding="utf-8",
    encoding_errors="ignore",
    on_bad_lines="skip",
)

make_all_graphs(master_df)
